In [1]:
import requests
import pandas as pd
import config

In [2]:
api_key = config.api_key 
bbox = '-6.15450317,53.30408682,-6.07896861,53.34080955'
satellites = 'Sentinel-1A, Sentinel-1B, Sentinel-2A, Sentinel-2B, Sentinel-3A, Sentinel-3B,Landsat-8'

url = 'https://api.spectator.earth/overpass/?api_key={api_key}&bbox={bbox}&satellites={satellites}'.format(
    api_key=api_key, bbox=bbox, satellites=satellites)

response = requests.get(url)
data = response.json()

In [3]:
result = []
for i in range(len(data['overpasses'])):
  overpass = {}
  overpass['date'] = data['overpasses'][i]['date']
  overpass['satellite'] = data['overpasses'][i]['satellite']
  print(overpass)
  result.append(overpass)

{'date': '2023-10-18T18:06:08.934369Z', 'satellite': 'Sentinel-1B'}
{'date': '2023-10-18T21:01:50.220736Z', 'satellite': 'Sentinel-3A'}
{'date': '2023-10-18T22:03:14.783321Z', 'satellite': 'Sentinel-3B'}
{'date': '2023-10-19T07:24:08.934369Z', 'satellite': 'Sentinel-1B'}
{'date': '2023-10-19T10:49:50.220736Z', 'satellite': 'Sentinel-3A'}
{'date': '2023-10-19T11:36:08.034907Z', 'satellite': 'Sentinel-2B'}
{'date': '2023-10-19T17:18:36.820365Z', 'satellite': 'Sentinel-1A'}
{'date': '2023-10-19T21:37:14.783321Z', 'satellite': 'Sentinel-3B'}
{'date': '2023-10-19T21:52:19.274965Z', 'satellite': 'Landsat-8'}
{'date': '2023-10-19T22:10:41.506121Z', 'satellite': 'Sentinel-2A'}
{'date': '2023-10-20T06:47:36.820365Z', 'satellite': 'Sentinel-1A'}
{'date': '2023-10-20T07:43:08.934369Z', 'satellite': 'Sentinel-1B'}
{'date': '2023-10-20T10:23:50.220736Z', 'satellite': 'Sentinel-3A'}
{'date': '2023-10-20T11:22:19.274965Z', 'satellite': 'Landsat-8'}
{'date': '2023-10-20T11:25:14.783321Z', 'satellite':

In [4]:
data = pd.DataFrame(result, columns=['satellite', 'date'])
data['Day'] = pd.to_datetime(data['date']).dt.day_name()
data['Date'] = pd.to_datetime(data['date']).dt.date
data['IST'] = pd.DatetimeIndex(pd.to_datetime(data['date'])).tz_convert('Europe/Dublin')
data['UTC'] = pd.to_datetime(data['date']).dt.time
data['IST'] = pd.to_datetime(data['IST']).dt.time
data.pop('date')
data = data.rename(columns={'satellite': 'Satellite'})

In [5]:
data['Date'] = data['Date'].astype(str)
data['IST'] = data['IST'].astype(str)
data['UTC'] = data['UTC'].astype(str)

In [6]:
data.head()

,Satellite,Day,Date,IST,UTC
0,Sentinel-1B,Wednesday,2023-10-18,19:06:08.934369,18:06:08.934369
1,Sentinel-3A,Wednesday,2023-10-18,22:01:50.220736,21:01:50.220736
2,Sentinel-3B,Wednesday,2023-10-18,23:03:14.783321,22:03:14.783321
3,Sentinel-1B,Thursday,2023-10-19,08:24:08.934369,07:24:08.934369
4,Sentinel-3A,Thursday,2023-10-19,11:49:50.220736,10:49:50.220736


In [10]:
scopes = ['https://spreadsheets.google.com/feeds',
          'https://www.googleapis.com/auth/drive',]

credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Colab Notebooks/satelliteoverpasskey.json',scopes=scopes)
sheetid = '1hx5xmfN6G9hyoHuBxPAVrF75SU90OJ8ko_GVUAaGYds'

In [11]:
service = discovery.build('sheets', 'v4', credentials=credentials)
range_name="Sheet1!A1:D1"
body = {
    'majorDimension' : 'ROWS',
    'values': data.values.tolist()
}

In [ ]:
service.spreadsheets().values().append(
    spreadsheetId=sheetid, range=range_name, valueInputOption='USER_ENTERED',
    insertDataOption='INSERT_ROWS', body=body).execute()